In [2]:
!pip install nsepython

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from nsepython import *
import pandas as pd
from datetime import date
import datetime
days=20
filename =f"stock_nifty_next50_{days}.csv"
#filename ="stock_nifty50_365.csv"
#filename ="stock.csv"
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# nifty next 50
stocks = [
    "ABB", "ADANIENSOL", "ADANIGREEN", "ADANIPOWER", "AMBUJACEM", "BAJAJHLDNG",
    "BAJAJHFL", "BANKBARODA", "BPCL", "BRITANNIA", "BOSCHLTD", "CANBK", "CGPOWER",
    "CHOLAFIN", "DABUR", "DLF", "DMART", "GAIL", "GODREJCP", "HAVELLS",
    "HAL", "HYUNDAI", "ICICIGI", "ICICIPRULI", "INDHOTEL", "IOC", "INDIGO", "NAUKRI",
    "IRFC", "JINDALSTEL", "JSWENERGY", "LICI", "LODHA", "PIDILITIND", "PFC",
    "PNB", "RECLTD", "MOTHERSON", "SHREECEM", "SIEMENS", "SWIGGY", "TATAPOWER",
    "TORNTPHARM", "TVSMOTOR", "UNITDSPR", "VBL", "VEDL", "ZYDUSLIFE" ,"BDL","DATAPATTERN","HAL","MTARTECH"
]

print(stocks)

['ABB', 'ADANIENSOL', 'ADANIGREEN', 'ADANIPOWER', 'AMBUJACEM', 'BAJAJHLDNG', 'BAJAJHFL', 'BANKBARODA', 'BPCL', 'BRITANNIA', 'BOSCHLTD', 'CANBK', 'CGPOWER', 'CHOLAFIN', 'DABUR', 'DLF', 'DMART', 'GAIL', 'GODREJCP', 'HAVELLS', 'HAL', 'HYUNDAI', 'ICICIGI', 'ICICIPRULI', 'INDHOTEL', 'IOC', 'INDIGO', 'NAUKRI', 'IRFC', 'JINDALSTEL', 'JSWENERGY', 'LICI', 'LODHA', 'PIDILITIND', 'PFC', 'PNB', 'RECLTD', 'MOTHERSON', 'SHREECEM', 'SIEMENS', 'SWIGGY', 'TATAPOWER', 'TORNTPHARM', 'TVSMOTOR', 'UNITDSPR', 'VBL', 'VEDL', 'ZYDUSLIFE', 'BDL', 'DATAPATTERN', 'HAL', 'MTARTECH']


In [4]:
# function definition
def my_equity_history_virgin(symbol,series,start_date,end_date):
    url="https://www.nseindia.com/api/historical/cm/equity?symbol="+symbol+"&series=[%22"+series+"%22]&from="+str(start_date)+"&to="+str(end_date)+""
    print("url is "+ url)
    payload = nsefetch(url)
    return pd.DataFrame.from_records(payload["data"])
def my_equity_history(symbol,series,start_date,end_date):
    #We are getting the input in text. So it is being converted to Datetime object from String.
    start_date = datetime.datetime.strptime(start_date, "%d-%m-%Y")
    end_date = datetime.datetime.strptime(end_date, "%d-%m-%Y")
    print("Starting Date: "+str(start_date))
    print("Ending Date: "+str(end_date))

    #We are calculating the difference between the days
    diff = end_date-start_date
    print("Total Number of Days: "+str(diff.days))
    print("Total FOR Loops in the program: "+str(int(diff.days/40)))
    print("Remainder Loop: " + str(diff.days-(int(diff.days/40)*40)))


    total=pd.DataFrame()
    for i in range (0,int(diff.days/40)):

        temp_date = (start_date+datetime.timedelta(days=(40))).strftime("%d-%m-%Y")
        start_date = datetime.datetime.strftime(start_date, "%d-%m-%Y")

        print("Loop = "+str(i))
        print("====")
        print("Starting Date: "+str(start_date))
        print("Ending Date: "+str(temp_date))
        print("====")

        #total=total.append(equity_history_virgin(symbol,series,start_date,temp_date))
        #total=total.concat(equity_history_virgin(symbol,series,start_date,temp_date))
        total = pd.concat([total, my_equity_history_virgin(symbol, series, start_date, temp_date)])


        print("Length of the Table: "+ str(len(total)))

        #Preparation for the next loop
        start_date = datetime.datetime.strptime(temp_date, "%d-%m-%Y")


    start_date = datetime.datetime.strftime(start_date, "%d-%m-%Y")
    end_date = datetime.datetime.strftime(end_date, "%d-%m-%Y")

    print("End Loop")
    print("====")
    print("Starting Date: "+str(start_date))
    print("Ending Date: "+str(end_date))
    print("====")

    #total=total.append(equity_history_virgin(symbol,series,start_date,end_date))
    #total=total.concat(equity_history_virgin(symbol,series,start_date,end_date))
    total = pd.concat([total, my_equity_history_virgin(symbol, series, start_date, end_date)])


    print("Finale")
    print("Length of the Total Dataset: "+ str(len(total)))
    payload = total.iloc[::-1].reset_index(drop=True)
    return payload

def fetch_equity_history_with_change(symbol, days, series="EQ"):
    end_date = datetime.datetime.now().strftime("%d-%m-%Y")
    start_date = (datetime.datetime.now() - datetime.timedelta(days=days)).strftime("%d-%m-%Y")
    print(start_date, end_date)
    df = my_equity_history(symbol, series, start_date, end_date)
    if df is None or df.empty:
        print(f"No data returned for {symbol}")
        return None           
    df["daily_change"] = df["CH_CLOSING_PRICE"].pct_change()
    # Uncomment below lines if you want to filter columns or drop first row
    # df = df[['CH_TIMESTAMP', 'daily_change']]
    # df = df.iloc[1:, :]
    #print(df)
    return df

#my_equity_history_virgin("M_M", "EQ","21-05-2025", "11-06-2025") 
#fetch_equity_history_with_change("M_M", days=20, series="EQ")

In [5]:
#download data
#linear execution 
#from random import randint
# import pandas as pd
# import datetime

# # Initialize the final table

# total = pd.DataFrame()

# # Define time range

# for symbol in stocks:
#     print(f"Fetching data for: {symbol}")
#     try:
#      total= total+fetch_equity_history_with_change(symbol, days=365, series="EQ")
#     except Exception as err:
#         print(f"Error: Skipping symbol {symbol} due to error: {err}")

# # Final review
# print("Final dataset preview:")
# print(total.head())

# parallel execution

from random import randint
import pandas as pd
import datetime
import concurrent.futures

# Initialize the final table
total = pd.DataFrame()

def fetch_wrapper(symbol):
    print(f"Fetching data for: {symbol}")
    try:
        return fetch_equity_history_with_change(symbol, days=days, series="EQ")
    except Exception as err:
        print(f"Error: Skipping symbol {symbol} due to error: {err}")
        return pd.DataFrame()  # Return empty DataFrame on error

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(fetch_wrapper, stocks))

# Combine all results
total = pd.concat(results, ignore_index=True)

# Final review
print("Final dataset preview:")
print(total.head())

Fetching data for: ABB
25-05-2025 14-06-2025
Starting Date: 2025-05-25 00:00:00
Ending Date: 2025-06-14 00:00:00
Total Number of Days: 20
Total FOR Loops in the program: 0
Remainder Loop: 20
End Loop
====
Starting Date: 25-05-2025
Ending Date: 14-06-2025
====
url is https://www.nseindia.com/api/historical/cm/equity?symbol=ABB&series=[%22EQ%22]&from=25-05-2025&to=14-06-2025
Fetching data for: ADANIENSOL
25-05-2025 14-06-2025
Starting Date: 2025-05-25 00:00:00
Ending Date: 2025-06-14 00:00:00
Total Number of Days: 20
Total FOR Loops in the program: 0
Remainder Loop: 20
End Loop
====
Starting Date: 25-05-2025
Ending Date: 14-06-2025
====
url is https://www.nseindia.com/api/historical/cm/equity?symbol=ADANIENSOL&series=[%22EQ%22]&from=25-05-2025&to=14-06-2025
Fetching data for: ADANIGREEN
25-05-2025 14-06-2025
Starting Date: 2025-05-25 00:00:00
Ending Date: 2025-06-14 00:00:00
Total Number of Days: 20
Total FOR Loops in the program: 0
Remainder Loop: 20
End Loop
====
Starting Date: 25-05-2

In [6]:
# save data
total.columns = [col.replace("CH_", "") for col in total.columns]
total.to_csv(f'../../inputdata/trading/{filename}',index=False)
#total.to_csv('./../inputdata/trading/total_may24_jun25.csv', sep='\t', index=False, header=True)
#payload.to_csv('./../inputdata/trading/total_may24_jun25_v2.csv', sep='\t', index=False, header=True)
#column_list=["CH_SYMBOL","CH_SERIES","CH_TIMESTAMP","TIMESTAMP","CH_TRADE_HIGH_PRICE","CH_TRADE_LOW_PRICE","CH_OPENING_PRICE","CH_CLOSING_PRICE","CH_LAST_TRADED_PRICE","CH_PREVIOUS_CLS_PRICE","CH_TOT_TRADED_QTY","CH_TOT_TRADED_VAL","CH_52WEEK_HIGH_PRICE","CH_52WEEK_LOW_PRICE","CH_TOTAL_TRADES","CH_ISIN","createdAt","updatedAt","SLBMH_TOT_VAL"]

#or use columns arg
#total.to_csv('../../inputdata/trading/total_niftyfifty.csv', columns = column_list,index=False)

In [7]:
import pandas as pd

# Load data
df = pd.read_csv(f"../../inputdata/trading/{filename}")
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df = df.sort_values(['SYMBOL', 'TIMESTAMP']).reset_index(drop=True)

results_df = []

# Process each symbol separately
for symbol in df['SYMBOL'].unique():
    sdf = df[df['SYMBOL'] == symbol].copy()
    sdf = sdf.sort_values('TIMESTAMP').reset_index(drop=True)
    
    # Calculate previous close and up days
    sdf['Prev_Close'] = sdf['CLOSING_PRICE'].shift(1)
    sdf['Up'] = sdf['CLOSING_PRICE'] > sdf['Prev_Close'] # Up by at least 1% 
    sdf['Up_1'] = sdf['Up'].shift(1)
    
    # 10-day average volume
    sdf['Avg_Vol_10'] = sdf['TOT_TRADED_QTY'].rolling(window=10).mean()
    
    # Buy signal: 2 consecutive up closes + volume surge (>1.2x avg volume)
    sdf['Buy_Signal'] = (sdf['Up'] & sdf['Up_1']) & (sdf['TOT_TRADED_QTY'] > 1.2 * sdf['Avg_Vol_10'])
  
    #sdf['Buy_Signal'] = (sdf['Up'] & sdf['Up_1']) & (sdf['TOT_TRADED_QTY'] > 1.2 * sdf['Avg_Vol_10']) & (sdf['TIMESTAMP']== (datetime.datetime.now().date - datetime.timedelta(days=1)).date())
    # Filter out rows with current date
    # print ("timestoand and date")
    # print (sdf['TIMESTAMP'])
    # print ("today date")
    yesterday= ((datetime.datetime.now()- datetime.timedelta(days=1)).date())

    profit = 0
    trades = 0
    for i, row in sdf[sdf['Buy_Signal']].iterrows():
        # Buy on the next day and hold for 3 days

        timestamp_date = pd.to_datetime(sdf.loc[i, 'TIMESTAMP']).date()
        total_traded_value = sdf.loc[i, 'TOT_TRADED_VAL']
        close_price = sdf.loc[i, 'CLOSING_PRICE']
        details = sdf.loc[i]
        # if(1==2):
        if(timestamp_date == yesterday)&(total_traded_value > 10000000)&(close_price > 50):
           # print(f"Skipping buy signal for {symbol} on {sdf.loc[i, 'TIMESTAMP']} as it is not yesterday's date {yesterday}")
            #print(f"Processing symbol: {symbol}, Buy signal at date {sdf.loc[i, 'TIMESTAMP']}")
            #results.append({'SYMBOL': symbol, 'DETAILS': details})
            results_df.append(details[['SYMBOL','CLOSING_PRICE', 'TIMESTAMP', 'TOT_TRADED_QTY', 'TOT_TRADED_VAL']])
    # else :  
    #     print(f"No trades for symbol {symbol}")
  
if  len(results_df) == 0:
    print("No profitable trades found.")
else : 
    # Sort by total profit descending
    results_df = pd.DataFrame(results_df)
    results_df = results_df.sort_values(by='TOT_TRADED_QTY', ascending=False)
    results_df.to_csv(f'../../output/trading/processed_{filename}', index=False)
    print(results_df)  # Top 10 symbols by profit
    print("Total trades found:", len(results_df))
    # best = results_df.iloc[0]
#print(f"\nMost profitable symbol: {best['SYMBOL']} with total profit ₹{round(best['Total Profit (₹)'],2)} from {best['Trades']} trades")
#sum_profit = results_df['Total Profit (₹)'].sum()



NameError: name 'results_df' is not defined